In [ ]:
import sys
sys.path.append('..')
from osp import *

df_meta = get_corpus_metadata()

In [ ]:
SLICE_LEN = 1000
slice_len = SLICE_LEN
stash = HashStash(f'osp_slices_{slice_len}')
stash

In [ ]:


def get_recog_words(txt):
    return [
        w.lower() for w in tokenize_agnostic(txt)
        if w.strip().isalpha() and w.lower() in get_ok_words()
    ]

def count_recog_words(txt, n=SLICE_LEN):
    return Counter(get_recog_words(txt.lower())[:n])

def get_text_freqs(id, slice_len=SLICE_LEN, force=False):
    stash = HashStash(f'osp_freqs_slices_{slice_len}')
    if not force and id in stash:
        return {int(k):v for k,v in stash[id].items()}
    slices = get_text_slices(id)
    freqs = {
        int(slice_num): dict(count_recog_words(txt, slice_len))
        for slice_num, txt in slices.items()
    }
    stash[id] = freqs
    return freqs

In [ ]:
# get_text_freqs('phil/10.2307/40231690',force=True)

In [ ]:
# count_recog_words('Hello, eee ex world!\n' * 1000)

In [ ]:
for id in tqdm(df_meta.index):
    get_text_freqs(id)


In [ ]:
STASH_SLICE_WORD_FREQS = HashStash('osp_freqs_slices_1000')
STASH_SLICE_WORD_FREQS

In [ ]:
# for x in STASH_SLICE_WORD_FREQS.items():
#     pass

In [ ]:
def iter_slice_word_freqs(df_meta=None):
    df_meta = get_corpus_metadata() if df_meta is None else df_meta
    for id in tqdm(df_meta.index):
        for slice_num, freqs in get_text_freqs(id).items():
            yield id, slice_num, freqs


In [ ]:
num_docs = Counter()
for id, slice_num, freqs in iter_slice_word_freqs():
    for w in freqs:
        num_docs[w] += 1

num_docs.most_common(10)

In [ ]:
num_docs_s = pd.DataFrame({'num_docs': num_docs}).rename_axis('word')
num_docs_s['num_docs_rank'] = num_docs_s['num_docs'].rank(method='min', ascending=False).apply(int)
num_docs_s.sort_values('num_docs_rank', ascending=True, inplace=True)
# num_docs_s.to_csv('../data/worddb_counts.csv')
num_docs_s

In [140]:
def get_words_freqs_slices(words, slice_len=SLICE_LEN):
    stash = HashStash(f'osp_word_freqs_slices_{slice_len}')
    # stash.clear()
    if not any(w not in stash for w in words):
        word2text2count = {
            w: stash[w]
            for w in words
        }
    else:        
        word2text2count = defaultdict(dict)
        for id, slice_num, freqs in iter_slice_word_freqs():
            for w, c in freqs.items():
                if w in words:
                    word2text2count[w][f'{id}__{slice_num:02d}'] = c

        for k,v in tqdm(list(word2text2count.items()), desc='saving to stash'):
            stash[k] = v

        for w in words:
            if w not in stash:
                word2text2count[w] = {}

    return pd.DataFrame(word2text2count).rename_axis('id__slice')

In [ ]:
# stash = HashStash(f'osp_word_freqs_slices_1000')
# stash

Config,Param,Value
Path,Root Dir,/Users/rj416/.cache/hashstash/osp_word_freqs_slices_1000
,Filename,data.db
Engine,Engine,lmdb
,Serializer,hashstash
,Compress,lz4
,B64,True
Stats,Len,641


In [144]:
words = get_non_content_words()
# words =['insofar']
df_freqs = get_words_freqs_slices(words)
df_freqs

  9%|▉         | 5124/57620 [00:24<04:13, 206.81it/s]

Unexpected exception formatting exception. Falling back to standard exception



Traceback (most recent call last):
  File "/Users/rj416/github/ordinary-style-philosophy/venv/lib/python3.11/site-packages/IPython/core/interactiveshell.py", line 3701, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/var/folders/d9/tm2sn8_12xsc6p6nfgtrd5h40000gp/T/ipykernel_84192/125031490.py", line 3, in <module>
    df_freqs = get_words_freqs_slices(words)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/var/folders/d9/tm2sn8_12xsc6p6nfgtrd5h40000gp/T/ipykernel_84192/1004263686.py", line None, in get_words_freqs_slices
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/rj416/github/ordinary-style-philosophy/venv/lib/python3.11/site-packages/IPython/core/interactiveshell.py", line 2201, in showtraceback
    stb = self.InteractiveTB.structured_traceback(
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/rj416/github/ordinary-style-philosophy/venv/lib/p

In [143]:
# df_freqs.sum(axis=1)

In [ ]:
num_docs_s.loc['insofar']

In [ ]:
non_content_words = get_non_content_words()
num_docs_s['is_non_content'] = num_docs_s.index.isin(non_content_words)
num_docs_s[num_docs_s.is_non_content].query('num_docs_rank < 10_000')